# Data understanding and Preprocessing

In [34]:
# importing all necessary libraries

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import numpy.random as nr
import math
import plotly.express as px
from sklearn import preprocessing
import sklearn.model_selection as ms
from sklearn import linear_model
import sklearn.metrics as sklm
import os

#To display graphics inline
%matplotlib inline

In [35]:

os.listdir()

['.git',
 '.ipynb_checkpoints',
 'Customer_Churn_Prediction.ipynb',
 'README.md',
 'sample_submission.csv',
 'Test.csv',
 'Train.csv',
 'VariableDefinitions.csv']

In [36]:
#load the dataset
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

In [37]:
train.head(2).T

,0,1
user_id,dcf68cc2fb515ccad7d8b9b3bd80ee2a4b270063,71c44b5ba328db5c4192a80f7cf8f244d9350ed0
REGION,SAINT-LOUIS,NaN
TENURE,K > 24 month,K > 24 month
MONTANT,17000,4300
FREQUENCE_RECH,32,29
REVENUE,18000,4427
ARPU_SEGMENT,6000,1476
FREQUENCE,34,37
DATA_VOLUME,NaN,1764
ON_NET,97,8


In [38]:
test.tail(2)

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,MRG,REGULARITY,TOP_PACK,FREQ_TOP_PACK
99998,afa76e894df4201fc77eb714de7d1f262299611a,THIES,K > 24 month,2500.0,4.0,2515.0,838.0,4.0,NaN,43.0,106.0,4.0,NaN,NaN,NO,16,All-net 500F=2000F;5d,5.0
99999,c08a2d84b87c1f5d4bb318114f508b77aa8e2663,MATAM,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NO,7,NaN,NaN


In [39]:
train.shape

(400000, 19)

In [40]:
test.shape

(100000, 18)

In [41]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400000 entries, 0 to 399999
Data columns (total 19 columns):
user_id           400000 non-null object
REGION            242480 non-null object
TENURE            400000 non-null object
MONTANT           259723 non-null float64
FREQUENCE_RECH    259723 non-null float64
REVENUE           265337 non-null float64
ARPU_SEGMENT      265337 non-null float64
FREQUENCE         265337 non-null float64
DATA_VOLUME       203146 non-null float64
ON_NET            254181 non-null float64
ORANGE            233683 non-null float64
TIGO              160614 non-null float64
ZONE1             31690 non-null float64
ZONE2             25513 non-null float64
MRG               400000 non-null object
REGULARITY        400000 non-null int64
TOP_PACK          232671 non-null object
FREQ_TOP_PACK     232671 non-null float64
CHURN             400000 non-null int64
dtypes: float64(12), int64(2), object(5)
memory usage: 58.0+ MB


In [42]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 18 columns):
user_id           100000 non-null object
REGION            60707 non-null object
TENURE            100000 non-null object
MONTANT           65049 non-null float64
FREQUENCE_RECH    65049 non-null float64
REVENUE           66510 non-null float64
ARPU_SEGMENT      66510 non-null float64
FREQUENCE         66510 non-null float64
DATA_VOLUME       50662 non-null float64
ON_NET            63617 non-null float64
ORANGE            58800 non-null float64
TIGO              40212 non-null float64
ZONE1             7680 non-null float64
ZONE2             6422 non-null float64
MRG               100000 non-null object
REGULARITY        100000 non-null int64
TOP_PACK          58297 non-null object
FREQ_TOP_PACK     58297 non-null float64
dtypes: float64(12), int64(1), object(5)
memory usage: 13.7+ MB


In [43]:
train.describe()

,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,ZONE1,ZONE2,REGULARITY,FREQ_TOP_PACK,CHURN
count,259723.000000,259723.000000,265337.000000,265337.000000,265337.000000,203146.000000,254181.000000,233683.000000,160614.000000,31690.000000,25513.000000,400000.000000,232671.000000,400000.00000
mean,5522.971346,11.503733,5505.487757,1835.167658,13.951835,3369.763441,275.917586,95.532927,23.134608,7.874282,7.187003,28.046502,9.254209,0.18711
std,7099.640630,13.275514,7175.802367,2391.929290,14.679943,12281.867504,873.593034,204.733272,63.061871,35.642843,26.964028,22.282773,12.305563,0.39000
min,20.000000,1.000000,1.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,0.00000
25%,1000.000000,2.000000,1000.000000,333.000000,3.000000,0.000000,5.000000,7.000000,2.000000,0.000000,0.000000,6.000000,2.000000,0.00000
50%,3000.000000,6.000000,3000.000000,1000.000000,9.000000,267.000000,27.000000,29.000000,6.000000,1.000000,2.000000,24.000000,5.000000,0.00000
75%,7300.000000,15.000000,7340.000000,2447.000000,19.000000,2925.000000,155.000000,99.000000,20.000000,3.000000,5.000000,51.000000,12.000000,0.00000
max,226550.000000,133.000000,233413.000000,77804.000000,91.000000,934576.000000,45011.000000,6788.000000,2758.000000,1657.000000,1011.000000,62.000000,629.000000,1.00000


### Finding and Treating Missing Values

In [44]:

train.isnull().sum()

user_id                0
REGION            157520
TENURE                 0
MONTANT           140277
FREQUENCE_RECH    140277
REVENUE           134663
ARPU_SEGMENT      134663
FREQUENCE         134663
DATA_VOLUME       196854
ON_NET            145819
ORANGE            166317
TIGO              239386
ZONE1             368310
ZONE2             374487
MRG                    0
REGULARITY             0
TOP_PACK          167329
FREQ_TOP_PACK     167329
CHURN                  0
dtype: int64

In [45]:
train.columns = train.columns.str.lower()

In [46]:
train.columns

Index(['user_id', 'region', 'tenure', 'montant', 'frequence_rech', 'revenue',
       'arpu_segment', 'frequence', 'data_volume', 'on_net', 'orange', 'tigo',
       'zone1', 'zone2', 'mrg', 'regularity', 'top_pack', 'freq_top_pack',
       'churn'],
      dtype='object')

In [56]:
#To get the percentage of missing values in each column

percent_missing = train.isnull().sum() * 100 / len(train)
print(percent_missing.round(2))

user_id            0.00
region            39.38
tenure             0.00
montant           35.07
frequence_rech    35.07
revenue           33.67
arpu_segment      33.67
frequence         33.67
data_volume       49.21
on_net            36.45
orange            41.58
tigo              59.85
zone1             92.08
zone2             93.62
mrg                0.00
regularity         0.00
top_pack          41.83
freq_top_pack     41.83
churn              0.00
dtype: float64
